In [20]:
import calcbench as cb
from IPython.core.display import display, HTML
import json
import itertools
from tqdm import tqdm, tqdm_notebook
import random
import urllib.error
import requests
import sys
from bs4 import BeautifulSoup

In [12]:
cb.api_client._rig_for_testing()

In [6]:
tenKQsections = cb.document_types()['tenKQSections']
disclosure_name_to_disclosure_type = {d['name'] : d['arcrole'] for d in tenKQsections}

In [9]:
len([s['name'] for s in tenKQsections])

18

In [16]:
len(bens_sections)

18

In [14]:
bens_sections = '''business_description
changes_in_and_disagreements_with_accountants
controls_and_procedures
corporate_governance
defaults_upon_senior_securities
disclosures_about_market_risk
executive_officers
legal_proceedings
management_s_discussion_and_analysis
market_for_registrants_common_equity
other_information
properties
relationships
report_of_independent_registered_public_accounting_firm
risk_factors
security_ownership
selected_financial_data
unresolved_sec_comments
'''.splitlines()

In [29]:
periods = [(year, period) for year in range(2010, 2018) for period in [0, 1, 2, 3]]

In [31]:
missing_docs = []
def get_documents_all_companies(disclosure_type, year, period):
    total_length = 0
    text_length = 0 
    try:
        found_documents = list(cb.document_search(entire_universe=True, document_type=disclosure_type, year=year, period=period))
    except requests.RequestException:
        print(disclosure_type, year, period)
    else:
        contents_length = 0
        docs = []
        for  doc in tqdm_notebook(found_documents, desc="{0} {1}-{2}".format(disclosure_type, year, period), leave=False):
            try:
                contents = doc.get_contents()
                total_length = total_length + len(contents)
                text_length += len(BeautifulSoup(contents, 'html.parser').text)
            except requests.RequestException:
                missing_docs.append(doc)
            except json.JSONDecodeError:
                print(disclosure_type, year, period, doc['ticker'])
    return total_length, text_length

In [30]:
test_doc_periods = list(itertools.product(tenKQsections, periods))

In [32]:
doc_period_sample = random.sample(test_doc_periods, 28)

In [32]:
doc_period_sample = [({'arcrole' : 2710}, (2015, 1))]

In [34]:
total_length = 0
for test_doc, period in tqdm_notebook(doc_period_sample, desc="period loop"):
    total_length, text_length = get_documents_all_companies(test_doc['arcrole'], year=period[0], period=period[1])

In [18]:
found_documents = list(cb.document_search(entire_universe=True, document_type=1110, year=2016, period=0))

In [21]:
contents = found_documents[0].get_contents()

In [25]:
soup = BeautifulSoup(contents, 'html.parser')

In [27]:
len(soup.text)

63659

In [28]:
len(contents)

135760

In [35]:
total_length

73157887

In [36]:
text_length

13967146

In [37]:
text_length / total_length

0.19091784321217478